In [2]:
import numpy as np
import pandas as pd
def split_int(inputs):
    inputs = inputs.split('=')[1]
    inputs = [int(i) for i in inputs.split('..')]    
    inputs.sort()
    return inputs

def do_m(inputs,mi):
    if mi[0] > min(inputs):
        mi[0] = min(inputs)
    if mi[1] < max(inputs):
        mi[1] = max(inputs)
    return mi

inputs = []
mx = [9,-9]
my = [9,-9]
mz = [9,-9]
with open("./input.txt","r") as file:
    for line in file:
        on_off = line.strip().split(' ')
        if on_off[0] == 'on':
            command = 1
        else:
            command = -1
        x,y,z = on_off[-1].split(',')
        x = split_int(x)
        y = split_int(y)
        z = split_int(z)
        inputs.append({
            'c' : command,
            'xMin' : x[0],
            'xMax' : x[1],
            'yMin' : y[0],
            'yMax' : y[1],
            'zMin' : z[0],
            'zMax' : z[1],      
        })
        mx = do_m(x,mx)
        my = do_m(y,my)
        mz = do_m(z,mz)        
        
mx,my,mz = np.max(np.abs(mx)),np.max(np.abs(my)),np.max(np.abs(mz))
mx,my,mz

(98841, 97790, 99480)

In [3]:
cube = {}

for idx, i in enumerate(inputs):
    x = np.array([i['xMin'], i['xMax']])
    y = np.array([i['yMin'], i['yMax']])
    z = np.array([i['zMin'], i['zMax']])
    c = i['c']

    if x[1] < -50 or x[0] > 50 or y[1] < -50 or y[0] > 50 or z[1] < -50 or z[0] > 50:
        continue
        
    x = np.where(x < -50, -50, x)
    x = np.where(x > 50, 50, x)
    y = np.where(y < -50, -50, y)
    y = np.where(y > 50, 50, y)    
    z = np.where(y < -50, -50, z)
    z = np.where(y > 50, 50, z)        
    
    for xs in range(x[0], x[1]+1):
        for ys in range(y[0], y[1]+1):
            for zs in range(z[0], z[1]+1):            
                test_str = f"{xs},{ys},{zs}"
                if c == 1:
                    if test_str not in cube.keys():
                        cube[test_str] = 1
                else:
                    if test_str in cube.keys():
                        del cube[test_str]
len(cube)

590467

In [103]:
inputsDF = pd.DataFrame(inputs.copy())

In [108]:
b

c        1
xMin   -22
xMax    31
yMin   -48
yMax     6
zMin   -35
zMax     9
Name: 1, dtype: int64

In [107]:
a

c        1
xMin   -40
xMax     6
yMin   -36
yMax     9
zMin   -36
zMax    12
Name: 0, dtype: int64

In [110]:
a = inputsDF.iloc[0].copy()
b = inputsDF.iloc[1].copy()
c= None
# case 1 a != b - this gives 2 separate cubes
case1 = a['xMin']>b['xMax'] or a['yMin']>b['yMax'] or a['zMin']>b['zMax']

# case 2 a is inside b - this gives b
# written as "MIN <= X <= MAX"
case2x = (b['xMin']<=[a['xMin'],a['xMax']]).all() and ([a['xMin'],a['xMax']]<=b['xMax']).all()
case2y = (b['yMin']<=[a['yMin'],a['yMax']]).all() and ([a['yMin'],a['yMax']]<=b['yMax']).all()
case2z = (b['zMin']<=[a['zMin'],a['zMax']]).all() and ([a['zMin'],a['zMax']]<=b['zMax']).all()
case2 = case2x and case2y and case2z

# case 3 b is inside a - this gives a
case3x = (a['xMin']<=[b['xMin'],b['xMax']]).all() and ([b['xMin'],b['xMax']]<=a['xMax']).all()
case3y = (a['yMin']<=[b['yMin'],b['yMax']]).all() and ([b['yMin'],b['yMax']]<=a['yMax']).all()
case3z = (a['zMin']<=[b['zMin'],b['zMax']]).all() and ([b['zMin'],b['zMax']]<=a['zMax']).all()
case3 = case3x and case3y and case3z

# case 4 a min is outside but a max is inside b

# case 5 a min is inside but a max is outside b

# case 6 b min is outside but b max is inside a

# case 7 b min is inside but b max is outside a

if case1:
    print('yo')
    c=pd.DataFrame([a,b])
elif case2:
    print('yoyo')    
    # because B comes after A, and all of A is 
    # inside B, it does not matter whatever state 
    # A was in, it will become B
    c=pd.DataFrame(b)     
elif case3:
    print('yoyo')    
    
    # because B comes after A, but B is inside A there 
    # is a chance for 2 states to be returned
    
    c=pd.DataFrame(a)

    c.iloc['c']=

# case 2 a is part in, part out of b - this gives
#if ((a['xMax']>=[b['xMin'],b['xMax']]).sum()==1) or ((a['yMax']>=[b['yMin'],b['yMax']]).sum()==1) or ((a['zMax']>=[b['zMin'],b['zMax']]).sum()==1):
#    print('bugger')
    
c

In [43]:
(a['xMax']>=[b['xMin'], b['xMax']]).sum()==1

True

In [ ]:
import pandas as pd
cube = {}
df = pd.DataFrame(columns = ['x','y','z','c'])
for idx, row in enumerate(inputsDF.iloc()):


    xMin, xMax = row[['xMin','xMax']]
    yMin, yMax = row[['yMin','yMax']]
    zMin, zMax = row[['zMin','zMax']]
    c = row['c']
    X = np.arange(xMin,xMax+1)
    Y = np.arange(yMin,yMax+1)
    Z = np.arange(zMin,zMax+1)    
    data = [[x,y,z] for x in X for y in Y for z in Z]
    tempDF = pd.DataFrame(columns=['x','y','z'], data=data)
    print(f"{idx}-{len(df):,}-{len(tempDF):,}")
    tempDF['c']=c

    toMerge = pd.merge(df, tempDF, on=['x','y','z'], how='outer',indicator=True)
    toMerge['c']=0
    toMerge.loc[(toMerge['_merge']=='left_only'),'c']=toMerge[(toMerge['_merge']=='left_only')]['c_x']
    toMerge.loc[(toMerge['_merge']=='right_only'),'c']=toMerge[(toMerge['_merge']=='right_only')]['c_y']
    toMerge.loc[(toMerge['_merge']=='both'),'c']=toMerge[(toMerge['_merge']=='both')]['c_y']
    toMerge.drop('_merge',axis=1,inplace=True)
    toMerge.drop('c_x',axis=1,inplace=True)
    toMerge.drop('c_y',axis=1,inplace=True)
    toMerge.drop_duplicates(inplace=True)
    #len(toMerge)
    df = toMerge    
    
len(df[(df['c']==1)])

0-0-105,938
1-105,938-133,650
2-183,473-117,045
3-286,034-140,400
4-338,946-151,580
5-382,768-109,980
6-418,870-111,780
7-444,583-137,376
8-453,526-119,070
9-480,346-135,044
10-482,726-4,199
11-486,223-105,750
12-497,008-3,456
13-498,448-121,440
14-526,538-1,650
15-526,538-121,992
16-551,161-3,456
17-551,161-143,055
18-573,563-4,845
19-573,563-119,808


In [ ]:
1

In [ ]:
import pandas as pd
import itertools
cube = {}
df = pd.DataFrame(columns =['x','y','z'])


for idx, i in enumerate(inputs):
    print(idx+1, len(df))
    x = np.array(i['x'])
    y = np.array(i['y'])
    z = np.array(i['z'])
    c = i['c']      

    if c == 0:
        d2 = df[((df['x'] < x[0]) | (df['x'] > x[1]) |
            (df['y'] < y[0]) | (df['y'] > y[1]) | 
            (df['z'] < z[0]) | (df['z'] > z[1]))]
    else:
        xs = np.arange(x[0], x[1]+1)
        ys = np.arange(y[0], y[1]+1)
        zs = np.arange(z[0], z[1]+1)        
        df_new = pd.DataFrame(list(itertools.product(xs,ys,zs)),columns=['x','y','z'])
        df = pd.concat([df,df_new])
        #df = df.drop_duplicates()
    

In [ ]:
df[((df['x'] >= x[0]) & (df['x'] <= x[1]))]

In [ ]:
import pandas as pd
import itertools

xs = np.arange(x[0], x[1]+1)
ys = np.arange(y[0], y[1]+1)
zs = np.arange(z[0], z[1]+1)
df = pd.DataFrame(list(itertools.product(xs,ys,zs)),columns=['x','y','z'])
df[((df['x']==-20) & (df['y']==4) & (df['z']==-30))]